In [1]:
import numpy as np
from nalu import NALU

C:\Users\dsloetto\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
data = open('cnn.txt', 'r').read()
chars = list(set(data)) #set = get unique values
VOCAB_SIZE = len(chars)

In [3]:
print('chars:\n{}\n\nVOCAB_SIZE: {}'.format(chars, VOCAB_SIZE))

chars:
['B', '2', 'R', '+', 'z', 'j', '?', 'Y', ':', 'Ç', 'I', '”', 'ï', 'V', 'C', 't', 'Z', 'e', 'l', '.', 'í', 'K', '•', ')', 'X', ']', '5', 'x', '&', 'H', ';', 'P', 'c', 'v', 'T', '"', 'S', 'O', '$', 'ö', 'g', 'y', '0', 'A', '8', '3', 'd', 'F', 'u', 'k', 'm', 'q', 'é', 'f', 'G', 'D', 'à', 'ó', 'w', 'o', 'ñ', 'N', '9', 'L', '_', '1', '(', '@', 'n', '|', 'p', '’', '6', 'W', '–', 'ü', 'r', '“', '—', '%', '/', '-', ' ', 'U', '[', ',', 'J', '4', 'E', 'Q', 's', 'i', 'b', '7', 'a', 'h', "'", '\n', '!', 'M']

VOCAB_SIZE: 100


In [4]:
idx_to_char = {i:char for i, char in enumerate(chars)}
char_to_idx = {char:i for i, char in enumerate(chars)}

In [5]:
SEQ_LENGTH = 60 #input sequence length
N_FEATURES = VOCAB_SIZE  # For 1 hot encoding
N_SEQ = int(np.floor((len(data) - 1)/SEQ_LENGTH))

X = np.zeros((N_SEQ, SEQ_LENGTH, N_FEATURES))
y = np.zeros((N_SEQ, SEQ_LENGTH, N_FEATURES))

for i in range(N_SEQ):
    x_sequence = data[i * SEQ_LENGTH: (i+1) * SEQ_LENGTH]
    x_sequence_ix = [char_to_idx[c] for c in x_sequence]
    input_sequence = np.zeros((SEQ_LENGTH, N_FEATURES))
    for j in range(SEQ_LENGTH):
        input_sequence[j][x_sequence_ix[j]] = 1. # One hot encoding
    X[i] = input_sequence
    
    y_sequence = data[i * SEQ_LENGTH + 1: (i + 1) * SEQ_LENGTH + 1] # shifted 1 to the right
    y_sequence_ix = [char_to_idx[c] for c in y_sequence]
    target_sequence = np.zeros((SEQ_LENGTH, N_FEATURES))
    for j in range(SEQ_LENGTH):
        target_sequence[j][y_sequence_ix[j]] = 1. #again, one hot encoding
    y[i] = target_sequence

In [6]:
from  keras.models import Sequential
from keras.layers import LSTM, TimeDistributed, Dense, Activation

# try CuDNNLSTM on gpu

In [9]:
HIDDEN_DIM = 70 
LAYER_NUM = 2
NB_EPOCHS = 200
BATCH_SIZE = 128
VAL_SPLIT = 0.1

model = Sequential()
model.add(LSTM(HIDDEN_DIM,
               input_shape=(None, VOCAB_SIZE),
               return_sequences = True))

for _ in range(LAYER_NUM - 1):
    model.add(LSTM(HIDDEN_DIM, return_sequences=True))
model.add(TimeDistributed(Dense(VOCAB_SIZE)))
#model.add(Activation('NULA'))
model.add(Activation('relu'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['acc'])

In [10]:
def generate_text(model, length):
  ix = [np.random.randint(VOCAB_SIZE)]
  y_char = [idx_to_char[ix[-1]]]
  X = np.zeros((1, length, VOCAB_SIZE))
  for i in range(length):
    X[0, i, :][ix[-1]] = 1.
    ix = np.argmax(model.predict(X[:, :i+1,:])[0], 1)
    y_char.append(idx_to_char[ix[-1]])
  return ''.join(y_char)

In [11]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, Callback
# callback to save the model if better
filepath="tgt_model.hdf5"
save_model_cb = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
# callback to stop the training if no improvement
early_stopping_cb = EarlyStopping(monitor='val_loss', patience=0)
# callback to generate text at epoch end
class generateText(Callback):
    def on_epoch_end(self, batch, logs={}):
        print(generate_text(self.model, 100))
        
generate_text_cb = generateText()

callbacks_list = [save_model_cb, early_stopping_cb, generate_text_cb]

In [12]:
model.fit(X, y, batch_size=BATCH_SIZE, verbose=1, epochs=NB_EPOCHS, callbacks=callbacks_list, validation_split=VAL_SPLIT)

Train on 3656 samples, validate on 407 samples
Epoch 1/200
3656/3656 [==============================] - 8s 2ms/step - loss: 4.2127 - acc: 0.1146 - val_loss: 3.6082 - val_acc: 0.1572

Epoch 00001: val_acc improved from -inf to 0.15725, saving model to tgt_model.hdf5
L

e                                                                                                 
Epoch 2/200
3656/3656 [==============================] - 7s 2ms/step - loss: 3.5485 - acc: 0.1614 - val_loss: 3.4456 - val_acc: 0.1588

Epoch 00002: val_acc improved from 0.15725 to 0.15880, saving model to tgt_model.hdf5
o                                                                                                    
Epoch 3/200
3656/3656 [==============================] - 7s 2ms/step - loss: 3.4415 - acc: 0.1618 - val_loss: 3.3914 - val_acc: 0.1590

Epoch 00003: val_acc improved from 0.15880 to 0.15897, saving model to tgt_model.hdf5
]                                                                                     

In [13]:
from keras.models import load_model
filepath="tgt_model.hdf5"
model = load_model(filepath)

In [14]:
generate_text(model, 100)

'ïe an at an at an an at an an at an an at an an at an an at an an at an an at an an at an an at an an'